## Feature Selection ##

In [8]:
import pandas as pd
import ta
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from pandas.plotting import scatter_matrix
from statsmodels.tsa.stattools import grangercausalitytests
from sklearn.preprocessing import RobustScaler

In [9]:
# Import data 
df = pd.read_csv('../raw_data/20230609-150316-processed-data.csv')
df.head()

,timestamp,open,high,low,close,volume,volume_CMF,volume_FII,volume_MFI,volume_VWA,...,vortex_pos,S&P_close_filled,Interest_Rates_filled,Inflation_Rate_us,subhag_signal,Fisher,Fisher_trigger,supertrend,chop_index,Velocity
0,2019-09-08 20:00:00,10000.00,10000.00,10000.00,10000.00,0.002,NaN,NaN,NaN,NaN,...,NaN,2978.43,2.25,2.0,-1,0.00000,0.00000,1,inf,-1
1,2019-09-08 21:00:00,10000.00,10000.00,10000.00,10000.00,0.000,NaN,NaN,NaN,NaN,...,NaN,2978.43,2.25,2.0,-1,3.80020,0.00000,1,inf,1
2,2019-09-08 22:00:00,10344.77,10357.53,10337.43,10340.12,471.659,NaN,NaN,NaN,NaN,...,NaN,2978.43,2.25,2.0,-1,5.70030,3.80020,1,inf,1
3,2019-09-08 23:00:00,10340.12,10368.64,10334.54,10351.42,583.271,NaN,NaN,NaN,NaN,...,NaN,2978.43,2.25,2.0,-1,6.65035,5.70030,1,inf,1
4,2019-09-09 00:00:00,10351.42,10391.90,10324.77,10391.90,689.759,NaN,NaN,NaN,NaN,...,NaN,2978.43,2.25,2.0,-1,7.12538,6.65035,1,inf,1


In [10]:
df= df.dropna().reset_index(drop=True)
df

,timestamp,open,high,low,close,volume,volume_CMF,volume_FII,volume_MFI,volume_VWA,...,vortex_pos,S&P_close_filled,Interest_Rates_filled,Inflation_Rate_us,subhag_signal,Fisher,Fisher_trigger,supertrend,chop_index,Velocity
0,2019-09-11 19:00:00,9992.18,10110.21,9934.11,10044.87,643.320,-0.041563,154.369057,34.613137,10037.462370,...,0.900406,3000.93,2.25,2.00,-1,-0.33674,0.02920,-1,58.337947,-1
1,2019-09-11 20:00:00,10044.14,10077.34,10034.91,10054.43,645.218,-0.077929,1013.499774,38.128005,10036.146735,...,0.971911,3000.93,2.25,2.00,-1,-0.37469,-0.33674,-1,58.113691,-1
2,2019-09-11 21:00:00,10053.77,10123.29,10053.32,10068.20,608.631,-0.116032,2065.978217,46.729583,10039.019781,...,0.998304,3000.93,2.25,2.00,1,-0.08683,-0.37469,-1,58.966328,-1
3,2019-09-11 22:00:00,10069.21,10103.43,10043.71,10077.60,726.360,-0.117717,2746.236186,44.287748,10044.120556,...,1.058558,3000.93,2.25,2.00,1,0.16529,-0.08683,-1,57.693104,-1
4,2019-09-11 23:00:00,10073.64,10125.56,10064.11,10102.87,696.837,-0.070102,4869.498301,52.811412,10053.708535,...,1.169051,3000.93,2.25,2.00,1,0.58994,0.16529,1,62.394941,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34153,2023-06-03 20:00:00,27239.40,27246.40,27101.00,27147.60,15130.376,-0.048901,45306.477416,59.148253,27189.586323,...,0.983015,4282.37,5.25,5.08,1,0.55827,0.61369,-1,53.765071,1
34154,2023-06-03 21:00:00,27147.70,27173.70,27113.90,27138.80,6229.878,-0.075151,31002.276871,58.559862,27189.451257,...,1.015887,4282.37,5.25,5.08,-1,0.15512,0.55827,-1,54.075485,1
34155,2023-06-03 22:00:00,27138.90,27153.60,27000.00,27084.90,15389.644,-0.042819,-91926.878625,49.242500,27175.830184,...,0.892200,4282.37,5.25,5.08,-1,-0.36126,0.15512,-1,44.950464,1
34156,2023-06-03 23:00:00,27084.90,27119.50,27042.00,27052.70,7760.747,-0.045307,-114493.903593,43.575625,27168.759575,...,0.905507,4282.37,5.25,5.08,-1,-0.86198,-0.36126,-1,45.530993,1


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34158 entries, 0 to 34157
Data columns (total 61 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   timestamp                   34158 non-null  object 
 1   open                        34158 non-null  float64
 2   high                        34158 non-null  float64
 3   low                         34158 non-null  float64
 4   close                       34158 non-null  float64
 5   volume                      34158 non-null  float64
 6   volume_CMF                  34158 non-null  float64
 7   volume_FII                  34158 non-null  float64
 8   volume_MFI                  34158 non-null  float64
 9   volume_VWA                  34158 non-null  float64
 10  mom_AOI                     34158 non-null  float64
 11  mom_PPO                     34158 non-null  float64
 12  mom_PPO_signal              34158 non-null  float64
 13  mom_PVO                     341

In [12]:
train_ratio = 0.8

In [13]:
# splitting the data into test and train
split_index = int(len(df) * train_ratio)


train_data = df[:split_index]
test_data = df[split_index:]

In [14]:
scaler = RobustScaler()

In [15]:
X_train = train_data.drop(columns=["close", "timestamp"])

In [16]:
X_scaled = scaler.fit_transform(X_train)
X_scaled

array([[-0.38437438, -0.38313948, -0.38330925, ..., -1.        ,
         0.47344967, -1.        ],
       [-0.3827994 , -0.3841306 , -0.38023735, ..., -1.        ,
         0.45729795, -1.        ],
       [-0.38250751, -0.38274508, -0.3796763 , ..., -1.        ,
         0.51870806, -1.        ],
       ...,
       [-0.10266416, -0.10182003, -0.09863149, ...,  0.        ,
        -0.10054503,  0.        ],
       [-0.09969669, -0.10025209, -0.09772637, ...,  0.        ,
        -0.1381346 ,  0.        ],
       [-0.09913593, -0.10138885, -0.09643423, ...,  0.        ,
        -0.26970604,  0.        ]])

In [17]:
X_scaled=pd.DataFrame(X_scaled, columns=scaler.get_feature_names_out())

In [18]:
pd.concat([X_scaled, train_data[["timestamp"]]], axis=1)

,open,high,low,volume,volume_CMF,volume_FII,volume_MFI,volume_VWA,mom_AOI,mom_PPO,...,S&P_close_filled,Interest_Rates_filled,Inflation_Rate_us,subhag_signal,Fisher,Fisher_trigger,supertrend,chop_index,Velocity,timestamp
0,-0.384374,-0.383139,-0.383309,-0.778971,-0.415969,0.005950,-0.569517,-0.383508,-0.227920,-0.479761,...,-0.781578,2.0,1.166667,0.0,-0.194255,-0.030667,-1.0,0.473450,-1.0,2019-09-11 19:00:00
1,-0.382799,-0.384131,-0.380237,-0.778815,-0.642069,0.007004,-0.434184,-0.383548,-0.221014,-0.440672,...,-0.781578,2.0,1.166667,0.0,-0.211211,-0.194187,-1.0,0.457298,-1.0,2019-09-11 20:00:00
2,-0.382508,-0.382745,-0.379676,-0.781825,-0.878971,0.008296,-0.102997,-0.383461,-0.193666,-0.392258,...,-0.781578,2.0,1.166667,1.0,-0.082591,-0.211145,-1.0,0.518708,-1.0,2019-09-11 21:00:00
3,-0.382040,-0.383344,-0.379969,-0.772139,-0.889449,0.009130,-0.197015,-0.383306,-0.156656,-0.341229,...,-0.781578,2.0,1.166667,1.0,0.030059,-0.082515,-1.0,0.427006,-1.0,2019-09-11 22:00:00
4,-0.381905,-0.382677,-0.379347,-0.774568,-0.593408,0.011736,0.131172,-0.383015,-0.099536,-0.273727,...,-0.781578,2.0,1.166667,1.0,0.219799,0.030144,0.0,0.765650,-1.0,2019-09-11 23:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27321,-0.102243,-0.103910,-0.100381,0.615925,0.083984,0.927539,0.438193,-0.107689,0.791826,0.631412,...,-0.137841,3.0,2.000000,0.0,0.639013,0.693746,0.0,0.100019,0.0,2022-09-23 04:00:00
27322,-0.100815,-0.104440,-0.100234,0.095497,0.095196,0.675136,0.238532,-0.107815,0.826022,0.613002,...,-0.137841,3.0,2.000000,0.0,0.522109,0.639146,0.0,0.056243,0.0,2022-09-23 05:00:00
27323,-0.102664,-0.101820,-0.098631,0.676947,-0.044951,0.894535,0.659877,-0.106866,0.875611,0.639760,...,-0.137841,3.0,2.000000,0.0,0.440637,0.522232,0.0,-0.100545,0.0,2022-09-23 06:00:00
27324,-0.099697,-0.100252,-0.097726,0.770961,-0.399051,0.830400,1.119679,-0.105407,0.849811,0.662127,...,-0.137841,3.0,2.000000,0.0,0.456570,0.440754,0.0,-0.138135,0.0,2022-09-23 07:00:00


In [19]:
X_scaled

,open,high,low,volume,volume_CMF,volume_FII,volume_MFI,volume_VWA,mom_AOI,mom_PPO,...,vortex_pos,S&P_close_filled,Interest_Rates_filled,Inflation_Rate_us,subhag_signal,Fisher,Fisher_trigger,supertrend,chop_index,Velocity
0,-0.384374,-0.383139,-0.383309,-0.778971,-0.415969,0.005950,-0.569517,-0.383508,-0.227920,-0.479761,...,-0.537803,-0.781578,2.0,1.166667,0.0,-0.194255,-0.030667,-1.0,0.473450,-1.0
1,-0.382799,-0.384131,-0.380237,-0.778815,-0.642069,0.007004,-0.434184,-0.383548,-0.221014,-0.440672,...,-0.188319,-0.781578,2.0,1.166667,0.0,-0.211211,-0.194187,-1.0,0.457298,-1.0
2,-0.382508,-0.382745,-0.379676,-0.781825,-0.878971,0.008296,-0.102997,-0.383461,-0.193666,-0.392258,...,-0.059326,-0.781578,2.0,1.166667,1.0,-0.082591,-0.211145,-1.0,0.518708,-1.0
3,-0.382040,-0.383344,-0.379969,-0.772139,-0.889449,0.009130,-0.197015,-0.383306,-0.156656,-0.341229,...,0.235170,-0.781578,2.0,1.166667,1.0,0.030059,-0.082515,-1.0,0.427006,-1.0
4,-0.381905,-0.382677,-0.379347,-0.774568,-0.593408,0.011736,0.131172,-0.383015,-0.099536,-0.273727,...,0.775206,-0.781578,2.0,1.166667,1.0,0.219799,0.030144,0.0,0.765650,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27321,-0.102243,-0.103910,-0.100381,0.615925,0.083984,0.927539,0.438193,-0.107689,0.791826,0.631412,...,0.269669,-0.137841,3.0,2.000000,0.0,0.639013,0.693746,0.0,0.100019,0.0
27322,-0.100815,-0.104440,-0.100234,0.095497,0.095196,0.675136,0.238532,-0.107815,0.826022,0.613002,...,0.099401,-0.137841,3.0,2.000000,0.0,0.522109,0.639146,0.0,0.056243,0.0
27323,-0.102664,-0.101820,-0.098631,0.676947,-0.044951,0.894535,0.659877,-0.106866,0.875611,0.639760,...,0.639285,-0.137841,3.0,2.000000,0.0,0.440637,0.522232,0.0,-0.100545,0.0
27324,-0.099697,-0.100252,-0.097726,0.770961,-0.399051,0.830400,1.119679,-0.105407,0.849811,0.662127,...,0.774640,-0.137841,3.0,2.000000,0.0,0.456570,0.440754,0.0,-0.138135,0.0


In [20]:
## VIF Analysis
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

In [21]:
# compute VIF factor for feature index 0
vif(X_scaled.values, 0)

23165.384531257092

In [22]:
df=pd.DataFrame()

df["features"] = X_scaled.columns

df["vif_index"] = [vif(X_scaled.values, i) for i in range(X_scaled.shape[1])]

round(df.sort_values(by="vif_index", ascending = False),2)

/home/mehmet/.pyenv/versions/assist_trader/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,features,vif_index
16,mom_StochOscillator,inf
48,vortex_neg,inf
40,KST_sig,inf
39,KST_diff,inf
38,KST,inf
49,vortex_pos,inf
34,ARI_up,inf
33,ARI,inf
32,ARI_down,inf
47,vortex_ind_diff,inf


In [25]:
## Features we can rely on
valid_vif = df.query("vif_index < 10").features

In [26]:
len(valid_vif)

24

In [27]:
df.features.unique().shape

(59,)

In [28]:
X_scaled= pd.concat([X_scaled, train_data[["close"]]],axis=1)

In [29]:
# Compute correlation matrix
corr_matrix = X_scaled.corr()

In [30]:
close_corr= corr_matrix["close"].sort_values(ascending=False)

In [32]:
# Visualize the Corr matrix
plt.figure(figsize=(100, 100))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm")
plt.title("Correlation Matrix")
plt.show()

--> The heatmap visualization will display the correlation coefficients between each pair of features in your dataset. 
The colors in the heatmap represent the strength and direction of the correlation. 
Positive correlations are indicated by warmer colors (reds), 
negative correlations by cooler colors (blues), and no correlation by white.


In [34]:
# Compute autocorrelation
plot_acf(df['close'], lags=14)
plt.title("Autocorrelation Function")
plt.show()

KeyError: 'close'

In [ ]:
# Compute Partial Autocorrelation 
plot_pacf(data['close'], lags=14)
plt.title("Partial Autocorrelation Function")
plt.show()

In [ ]:
# Compute scatter matrix
subset = data["close"]

In [ ]:
scatter_matrix(subset, figsize=(10, 10), diagonal='kde')
plt.show()

### Granger Causality ###

In [ ]:
# put the feature you wanna analyze into subset
subset = data[["close", "volume"]]

In [ ]:
max_lag = 14  # Specify the maximum lag value for the test
results = grangercausalitytests(subset, max_lag)

-->  The p-values help evaluate the significance of the causality, where lower p-values indicate a stronger evidence of causality.

In [ ]:
for lag in range(1, max_lag + 1):
    p_value = results[lag][0]['ssr_chi2test'][1]
    print(f"Lag {lag}: p-value = {p_value}")